# JSON Validation Java Tutorial
This tutorial will show you how to validate a JSON document and create a customized string format tag using the Java classes in the com.ndi package. We will start this tutorial with a brief introduction to JSON Schema and the format tag.

## A brief introduction to JSON Schema and com.ndi package

For this project, JSON documents are used to represent instances of our ndi classes, which include ndi_session, ndi_daqreader, ndi_daqsystem and many others ndi classes. Every JSON file consists of a series of key-value pairs, each of which symbolically represents a class's field and its value. Therefore, before storing a JSON file into a database, it is crucial to ensure that the keys' values have the correct data type. To do so, we need to create a JSON schema document for each class that we wish to validate.

A JSON Schmea is special JSON file that is used to specify the expected data type for each value in a JSON document. We will provide a concrete example to further clarify our explanation. Say that we have an instance of ndi_subject which consits of two fields: **local_identifier** and **description**, we can represents this instance of ndi_subject with the following JSON file: *sample-ndi-subject.json*

<pre>
<code>
<b>sample-ndi-subject.json</b>

    {
        "local_identifier" : "subject@brandeis.edu",
        "description" : "this is a dummy subject"
    }
</code>
</pre>

Clearly both local_identifier and description have to be string, something like such will be invalid

<pre>
<code>
<b>sample-ndi-subject-wrong-type.json</b>

    {
        "local_identifier" : 153,
        "description" : "this is a dummy subject"
    }
</code>
</pre>

In order to enforce that both the field **local_identifier** and the field **description** to be string, we can create a JSON schema document, let's call it *ndi_document_subject_schema.json*

<pre>
<code>
<b>ndi_document_subject_schema.json</b>

    {
        "$schema": "http://json-schema.org/draft/2019-09/schema#",
        "id": "$NDISCHEMAPATH\/ndi_document_subject_schema.json",
        "title": "ndi_document_subject",
        "type": "object",
        "properties": {
            "local_identifier": {
                "type": "string"
            },
            "description": {
                "type": "string"
            }
        }
    }
</code>
</pre>

The **"$schema"** tag tells us the official json_schema specification we are using. You can read the specification document using this linke: "http://json-schema.org/draft/2019-09/schema#". The  **"id"** tag tells us the class of JSON document it is trying to enforce rules upon. The **"title"** tag specifies the name of the associated JSON document. All of the three above tags are semantic tags. That is, they don't have an impact on the validation outcome. 

The **"type"** tag specifies the expected data type of each value of the JSON document. Here *ndi_document_subject_schema.json* represents a MATLAB object, so we let the type to be "object". Next within the properties tag, we need to specify the expected data type of each fields of this object. Here we want both the fields "local_identifier" and "description" to be a string. You can read more about the key word an the expected structure of the JSON Schema document through this linke: "https://json-schema.org/understanding-json-schema


The classes within the package com.ndi, which can be found in ndi-validator-java.jar file, proivde methods that can be called from MATLAB to validate a JSON instance (in fact, this is precisely what *ndi_validate.m* does"). Particularly, the com.ndi.Validator class, which is a wrapper around org.everit's implementation of JSON Schema. You can check out their source code here: https://github.com/everit-org/json-schema/tree/master/core/src/main/java/org/everit/json/schema. We will explain how to use those methods in the next section of this tutorial.

## Validating JSON Document

Our next task is the use the Validator class in the com.ndi package to validate the JSON Document. First we need to import the Validator class from the com.ndi package. If you are curious, you can check out its implementation through this : https://github.com/VH-Lab/NDI-matlab/blob/document_validation_2/database/java/ndi-validator-java/src/main/java/com/ndi/Validator.java

In [1]:
import com.ndi.Validator;

There are two ways to construct a new instance of the Validator class. Here are the method signature of the Validaotr's constructor.

<pre>
<code>
<b>public</b> Validator(<b>String</b> document, <b>String</b> schema)

<b>public</b> Validator(<b>JSONObject</b> document, <b>JSONObject</b> schema)
</code>
</pre>

**The first constructor takes two parameters**:

<ul>
    <li> <b> document</b>: this represents the content of the JSON document we wish to validate</li>
    <li> <b> schema</b>: this represents the content of the JSON schema document we wish to validate the document against</li>
</ul>

**The second constructor also takes two parameters**:
<ul>
    <li> <b> document</b>: same as what the first constructor takes, except document needs to be an instance of org.json.JSONObject</li>
    <li> <b> schema</b>: again, same as what constructor takes, but JSON document needs to be wrapped inside org.json.JSONObject</li>
</ul>

An example will make will hopefully makes our explaination clear. Let's try to validate the *sample-ndi-subject.json* we have created above against the *ndi_document_subject_schema.json*. We will construct a Validator object using the first constructor first:

In [2]:
//the file content of the sample-ndi-subject.json

String document = "{\n" +
                "    local_identifier : \"subject@brandeis.edu\",\n" +
                "    description : \"this is a dummy subject\"\n" +
                "}";

In [3]:
//the file content of the ndi_document_subject_schema.json

String schema = "{\n" +
            "    \"$schema\": \"http://json-schema.org/draft/2019-09/schema#\",\n" +
            "    \"id\": \"$NDISCHEMAPATH\\/ndi_document_subject_schema.json\",\n" +
            "    \"title\": \"ndi_document_subject\",\n" +
            "    \"type\": \"object\",\n" +
            "    \"properties\": {\n" +
            "        \"local_identifier\": {\n" +
            "            \"type\": \"string\"\n" +
            "        },\n" +
            "        \"description\": {\n" +
            "            \"type\": \"string\"\n" +
            "        }\n" +
            "    }\n" +
            "}";

document and schema represent the actual JSON content as string

In [4]:
Validator ndi_subject = new Validator(document, schema);

Next we will call the Validator's instance method getReport() to get a detailed report of our validation. This method returns an instance of java.util.HashMap, which tells you the part of the JSON document that has a type error.

In [5]:
ndi_subject.performValidation()

{}

We get an empty HashMap, which means that our JSON document does not contain any type error. Both the local_identifier field and the description field were initialized to be a string. This confirms that our JSON document is valid. Next, let's see what happen if we entered an invalid value to initialize one of our fields:

In [6]:
//the file content of the sample-ndi-subject-wrong-type.json

String document = "{\n" +
                "    local_identifier : 153,\n" +
                "    description : \"this is a dummy subject\"\n" +
                "}";

In [7]:
Validator ndi_subject = new Validator(document, schema);
ndi_subject.getReport();

{#/local_identifier=#/local_identifier: expected type: String, found: Integer}

We got an instance of HashMap, which tells us that the field "local_identifier" is a Integer, which is supposed to be a string based on the schema document we've passed in. Our validation fails just as what we would have expected. We can also validate our JSON document by passing in an instance of org.JSONObject instead of a string. The reason we have a second constructor is that we can initialize a JSONObject by passing in the file path to the JSON document as opposed to its content:

To create an instance of JSONObject from a file path, we need to wrap the file path inside a InputFileStream object. Then wrap that InputFileStream object inside a JSONTokner Object, and finally pass that JSONTokener object to the JSONObject constructor. Suppose we have created the files "sample-ndi-subject-wrong-type.json" and "ndi_document_subject_schema.json" (yes they are the exact same file we have used in our earlier example) in our java classpath. We use a try-with-resource block to safely read those files. To learn more about Java's try-with resource block and IO syntax, check out this link: https://docs.oracle.com/javase/tutorial/essential/exceptions/tryResourceClose.html. 

In [8]:
import org.json.JSONObject;
import org.json.JSONTokener;
import java.io.FileInputStream;
import java.io.InputStream;

JSONObject document;
JSONObject schema;

try(InputStream schemaFile = new FileInputStream("/Users/yixin/Documents/ndi-validator-java/src/main/resources/sample-ndi-subject-wrong-type.json");
    InputStream documentFile = new FileInputStream("/Users/yixin/Documents/ndi-validator-java/src/main/resources/ndi_document_subject_schema.json")){
    document = new JSONObject(new JSONTokener(documentFile));
    schema = new JSONObject(new JSONTokener(schemaFile));
}

Verify that we have successfully loaded the JSON file from disk

In [9]:
document

{"$schema":"http://json-schema.org/draft/2019-09/schema#","id":"$NDISCHEMAPATH/ndi_document_subject_schema.json","title":"ndi_document_subject","type":"object","properties":{"local_identifier":{"type":"string"},"description":{"type":"string"}}}

In [10]:
schema

{"local_identifier":153,"description":"this is a dummy subject"}

Next, we just pass those instances of JSONObject into our constructor:

In [11]:
Validator validator = new Validator(schema, document);
validator.getReport();

{#/local_identifier=#/local_identifier: expected type: String, found: Integer}

We've got what we would have expected, an error message identifying the appropriate type mismatch error. You've got through the first part of the tutorial successfully. Next we will discuss how we can define our own JSON Schema vocabulary (the format keyword). 

## Creating your own format keyword within the string type

In the previous section, we saw how we could restrict the values of a JSON document to a data type. However, what if we want our string to be in a specific format. For instance, what if we want our value to be a valid email address. The official JSON Schema specification allows us to use the format tag to enforce the string to contains a particular pattern. In fact, the official JSON Schema Specification offers the following built-in formats including:

<ul>
    <li>"date-time"</li>
    <li>"email"</li>
    <li>"hostname"</li>
    <li>"ipv4"</li>
    <li>"ipv6"</li>
    <li>"uri"</li>
</ul>

See the full list here: https://json-schema.org/understanding-json-schema/reference/string.html#format

To make our explination clearer, let's try to validate a JSON document, one of whose value is supposed to be an email address

<pre>
<code>
<b>studentA.json</b>

    {
        "name" : "studentA",
        "email-address" : "studentA@brandeis.edu"
    }
</code>
</pre>

<pre>
<code>
<b>studentB.json</b>

    {
        "name" : "studentB",
        "email-address" : "badEmailAddress!%^@"
    }
</code>
</pre>

<pre>
<code>
<b>student_schema.json</b>

    {
        "$schema": "http://json-schema.org/draft/2019-09/schema#",
        "id": "$NDISCHEMAPATH\/student_schema.json",
        "title": "student",
        "type": "object",
        "properties": {
            "name": {
                "type": "string"
            },
            "email-address": {
                "type": "string",
                "format" : "email"
            }
        }
    }
</code>
</pre>

As you can see, we add a "format" tag in this student_schema.json for the field "email-address". Our validator will not only check if the value is type string but also verify if this is indeed a valid email address. We will run both studentA.json and studentB.json against our student_schema.json. 

In [12]:
//the file content of studentA.json
String studentA = "{\n" +
                    "\"name\" : \"studentA\",\n" +
                    " \"email-address\" : \"studentA@brandeis.edu\"\n" +
                    "}";

In [13]:
//the file content of studentB.json
String studentB = " {\n" +
                  " \"name\" : \"studentB\",\n" +
                  " \"email-address\" : \"badEmailAddress!%^@\"\n" +
                  " }";

In [14]:
//the file content of student_schema.json
String studentSchema =  "{\n" +
                        "        \"$schema\": \"http://json-schema.org/draft/2019-09/schema#\",\n" +
                        "        \"id\": \"$NDISCHEMAPATH\\/student_schema.json\",\n" +
                        "        \"title\": \"student\",\n" +
                        "        \"type\": \"object\",\n" +
                        "        \"properties\": {\n" +
                        "            \"name\": {\n" +
                        "                \"type\": \"string\"\n" +
                        "            },\n" +
                        "            \"email-address\": {\n" +
                        "                \"type\": \"string\",\n" +
                        "                \"format\" : \"email\"\n" +
                        "            }\n" +
                        "        }\n" +
                        "    }";

In [15]:
// initialize our validators
Validator validatorForStudentA = new Validator(studentA, studentSchema);
Validator validatorForStudentB = new Validator(studentB, studentSchema);

In [16]:
validatorForStudentA.getReport()

{}

In [17]:
validatorForStudentB.getReport()

{#/email-address=#/email-address: [badEmailAddress!%^@] is not a valid email address}

Just as what we would have expected, our validator is capable of detecting invalid email address. Next what if we want our string to be a particular format that the JSON Schema specification does not have. We can implement this logic by ourselves. This requires us to implements the org.everit.json.schema.FormatValidator interface. We will demonstrate how to achieve that through a concrete example. Suppose we want our validator to only accept email address that comes from @brandeis.edu email address. That is, "@" must be followed by "brandeis.edu".

Let's create a class called BrandeisEmailValidator that implements the org.everit.json.schema.FormatValidator interface:

In [18]:
import org.everit.json.schema.FormatValidator;

public class BrandeisEmailValidator implements FormatValidator{
    
    @Override
    public Optional<String> validate(String subject){
        int separator = subject.indexOf("@");
        if (separator == -1 || !subject.substring(separator).equals("@brandeis.edu")){
            return Optional.of("requires a brandeis.edu email");
        }
        return Optional.empty();
    }
    
    @Override
    public String formatName(){
        return "brandeis-email";
    }
}

<p>The two methods we have to override are the public Optional&lt;String&gt; validate(String subject) method, which takes an user input and check if it is a valid Brandeis email address, and another method: public String formatName(), which simply returns the format keyword that we want the validator to recognize when it parses the json schema document. As the method signature suggests, the first method needs to return a string wrapped inside the Optional container object. If something went wrong, the string will represent the error message, otherwise we return an empty Optional container object to indicate that no type error has found. Let's test it out. Modify the email address in studentA.json and studentB.json. Only student B has a valid brandeis email address</p>

<pre>
<code>
<b>studentA.json</b>

    {
        "name" : "studentA",
        "email-address" : "studentA@bu.edu"
    }
</code>
</pre>

<pre>
<code>
<b>studentB.json</b>

    {
        "name" : "studentB",
        "email-address" : "studentB@brandeis.edu"
    }
</code>
</pre>

<pre>
<code>
<b>student_schema.json</b>

    {
        "$schema": "http://json-schema.org/draft/2019-09/schema#",
        "id": "$NDISCHEMAPATH\/student_schema.json",
        "title": "student",
        "type": "object",
        "properties": {
            "name": {
                "type": "string"
            },
            "email-address": {
                "type": "string",
                "format" : "brandeis-email"
            }
        }
    }
</code>
</pre>

In [19]:
//the file content of studentA.json
String studentA = "{\n" +
                    "\"name\" : \"studentA\",\n" +
                    " \"email-address\" : \"studentA@bu.edu\"\n" +
                    "}";

In [20]:
//the file content of studentB.json
String studentB = " {\n" +
                  " \"name\" : \"studentB\",\n" +
                  " \"email-address\" : \"studentB@brandeis.edu\"\n" +
                  " }";

In [21]:
//the file content of student_schema.json
String studentSchema =  "{\n" +
                        "        \"$schema\": \"http://json-schema.org/draft/2019-09/schema#\",\n" +
                        "        \"id\": \"$NDISCHEMAPATH\\/student_schema.json\",\n" +
                        "        \"title\": \"student\",\n" +
                        "        \"type\": \"object\",\n" +
                        "        \"properties\": {\n" +
                        "            \"name\": {\n" +
                        "                \"type\": \"string\"\n" +
                        "            },\n" +
                        "            \"email-address\": {\n" +
                        "                \"type\": \"string\",\n" +
                        "                \"format\" : \"brandeis-email\"\n" +
                        "            }\n" +
                        "        }\n" +
                        "    }";

In [22]:
// initialize our validators
Validator validatorForStudentA = new Validator(studentA, studentSchema);
Validator validatorForStudentB = new Validator(studentB, studentSchema);

This time, we need to add our BrandeisEmailValidator class we've just written to the Validator object so that our Validator class knows which methods to call when it sees our pre-defined foramt tag "brandeis-email" while scanning through the schema document. This can be done through calling Validator <b>addValidator()</b> method, which returns a new instance of the Validator class with the FormatValidator added to it. 

In [23]:
validatorForStudentA = validatorForStudentA.addValidator(new BrandeisEmailValidator());
validatorForStudentB = validatorForStudentB.addValidator(new BrandeisEmailValidator());

Now let's check if our Validator is able to tell if our student has a Brandeis email address

In [24]:
validatorForStudentA.getReport()

{#/email-address=#/email-address: requires a brandeis.edu email}

In [25]:
validatorForStudentB.getReport()

{}

Our validator knows if a student has a valid Brandeis email address. This completes the tutorial. Now you know how to use com.ndi.Validate to validate a JSON document and how to write your own JSON format keyword. Remeber that all the classes in the com.ndi packages are automatically added to the MATLAB javapath after ndi_Init is run, so you have access to all the classes in the package and their methods. Simply import the package and call them from MATLAB if you ever need them. 